# Considerations of vehicle specific behaviors

This figure shows examples of vehicle level peculiarities that can affect the proposed standard measurment. \
* measurements of five Cupra Borns show different voltages, due to cell-to-cell varations or differnce in estimations from the onboard SOC estimator --> voltage window must be carefully defined to fit all vehicles of a model type
* software updates, possibly over-the-air, can alter the behavior of the battery pack, e.g. the usable voltage window --> changes must be transparently communicated to the customer and the standard measurement adapted accordingly
* BMS balancing is (regularly) necessary to align voltage deviations of the cells in the pack. The proposed standard measurement might not be reproducible if the cells are imbalanced --> battery pack shall be in a balanced state prior to the measurement
* faulty cells in a pack define the overall capacity/energy retention --> even if the standard measurement fails, as cut-off voltages cannot be met, battery diagnosis is possible to locate faulty cells, if cell voltage data is available, which the legislator must enforce

In [ ]:
import os
import sys
import math
sys.path.append(os.path.join(os.getcwd().partition('nature_soh')[0], "nature_soh"))
from src.config_base import GeneralConfig

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter
import matplotlib.gridspec as gridspec
from matplotlib.ticker import AutoMinorLocator, FixedLocator
import matplotlib.patches as patches
import matplotlib.patheffects as mpe

In [ ]:
from src.data.read_feather import ReadFeather
from src.filtering.filter_methods import FilterMethods
from src.voltage_capacity_analysis.dva import DVA
from src.visualization.config_visualization import VisualizationConfig, instantiate_matplotlib_parameters, cm2inch
instantiate_matplotlib_parameters()
from src.visualization.colormaps import ColorMaps

In [ ]:
tum_orange_cmap = ColorMaps.orange_tum()
orange_colors = tum_orange_cmap(np.linspace(0,1, 10))
tum_orange_cmap

In [ ]:
tum_blue_cmap = ColorMaps.blue_tum()
blue_colors = tum_blue_cmap(np.linspace(0,1, 10))
tum_blue_cmap

In [ ]:
tum_cmap = ColorMaps.blue_orange_tum(grayscale=0.5)
colors = tum_cmap(np.linspace(0,1, 5))
tum_cmap

## Load Data

In [ ]:
def filter_func_preprocess(signal):
    perc_filter = 1/100 * len(signal) #% filter
    filtered_signal = FilterMethods().rolling_mean_df(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter))
    #filtered_signal = FilterMethods().savgol(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter * len(signal)))
    return filtered_signal

In [ ]:
read_feather = ReadFeather()
read_feather.set_filter_U(filter_func_preprocess)
read_feather.set_filter_Q(filter_func_preprocess)

### model 2 model varation
Nr in file name corresponds to the license plate number

In [ ]:
df_cupra_288 =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"Cupra","Cupra_Born_288_JB_8A_CEE7_CS_2024.feather"))

In [ ]:
df_cupra_397 =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"Cupra","Cupra_Born_397_JB_8A_CEE7_CS_2024.feather"))

In [ ]:
df_cupra_349 =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"Cupra","Cupra_Born_349_JB_8A_CEE7_CS_2024.feather"))

In [ ]:
df_cupra_204 =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"Cupra","Cupra_Born_204_JB_8A_CEE7_CS_2024.feather"))

In [ ]:
df_cupra_213 =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"Cupra","Cupra_Born_213_JB_8A_CEE7_CS_2024.feather"))

Cupra voltage spread

In [ ]:
%%capture
df_cupra_397["E"] = sp.integrate.cumtrapz((df_cupra_397["U"].values*df_cupra_397["I"].values)/1000, df_cupra_397["time_h"], initial=0)
df_cupra_349["E"] = sp.integrate.cumtrapz((df_cupra_349["U"].values*df_cupra_349["I"].values)/1000, df_cupra_349["time_h"], initial=0)
df_cupra_288["E"] = sp.integrate.cumtrapz((df_cupra_288["U"].values*df_cupra_288["I"].values)/1000, df_cupra_288["time_h"], initial=0)
df_cupra_204["E"] = sp.integrate.cumtrapz((df_cupra_204["U"].values*df_cupra_204["I"].values)/1000, df_cupra_204["time_h"], initial=0)
df_cupra_213["E"] = sp.integrate.cumtrapz((df_cupra_213["U"].values*df_cupra_213["I"].values)/1000, df_cupra_213["time_h"], initial=0)

In [ ]:
%%capture
df_cupra_397["SOC_Q"] = df_cupra_397["Q"].values/df_cupra_397["Q"].max()*100
df_cupra_349["SOC_Q"] = df_cupra_349["Q"].values/df_cupra_349["Q"].max()*100
df_cupra_288["SOC_Q"] = df_cupra_288["Q"].values/df_cupra_288["Q"].max()*100
df_cupra_204["SOC_Q"] = df_cupra_204["Q"].values/df_cupra_204["Q"].max()*100
df_cupra_213["SOC_Q"] = df_cupra_213["Q"].values/df_cupra_213["Q"].max()*100

In [ ]:
df_U_spread = df_cupra_397[["SOC","SOC_Q","U"]].copy()
df_U_spread = pd.merge_asof(df_U_spread, df_cupra_349[["SOC_Q","U"]].copy(), on=['SOC_Q'], direction='nearest',suffixes=('_397', '_349'))
df_U_spread = pd.merge_asof(df_U_spread, df_cupra_288[["SOC_Q","U"]].copy(), on=['SOC_Q'], direction='nearest',suffixes=('', '_288'))
df_U_spread = pd.merge_asof(df_U_spread, df_cupra_204[["SOC_Q","U"]].copy(), on=['SOC_Q'], direction='nearest',suffixes=('', '_204'))
df_U_spread = pd.merge_asof(df_U_spread, df_cupra_213[["SOC_Q","U"]].copy(), on=['SOC_Q'], direction='nearest',suffixes=('', '_213'))
df_U_spread.rename(columns={"U":"U_288"},inplace=True)

In [ ]:
df_U_spread["U_mean"] = df_U_spread[["U_204","U_213","U_288","U_397","U_349"]].mean(axis=1)
df_U_spread["U_min"] = df_U_spread[["U_204","U_213","U_288","U_397","U_349"]].min(axis=1)
df_U_spread["U_max"] = df_U_spread[["U_204","U_213","U_288","U_397","U_349"]].max(axis=1)
df_U_spread["dU_min"] = df_U_spread["U_min"]-df_U_spread["U_mean"]
df_U_spread["dU_max"] = df_U_spread["U_max"]-df_U_spread["U_mean"]

### software update

In [ ]:
df_vw =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"VW","VW_ID3_JB_8A_CEE7_FTM_2021.feather"))

In [ ]:
df_vw_2 =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"VW","VW_ID3_JB_8A_CEE7_FTM_2023.feather"))

In [ ]:
df_vw_3 =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"VW","VW_ID3_JB_8A_CEE7_FTM_2024.feather"))

In [ ]:
%%capture
df_vw["E"] = sp.integrate.cumtrapz((df_vw["U"].values*df_vw["I"].values)/1000, df_vw["time_h"], initial=0)
df_vw_2["E"] = sp.integrate.cumtrapz((df_vw_2["U"].values*df_vw_2["I"].values)/1000, df_vw_2["time_h"], initial=0)
df_vw_3["E"] = sp.integrate.cumtrapz((df_vw_3["U"].values*df_vw_3["I"].values)/1000, df_vw_3["time_h"], initial=0)

## BMS balancing

In [ ]:
df_vw_relax =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"VW","VW_ID3_FTM_JB_8A_CEE7_FTM_2024_balanced.feather"))

In [ ]:
df_vw_balance =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"VW","VW_ID3_FTM_JB_8A_CEE7_FTM_2024_imbalanced.feather"))

### cell defect

In [ ]:
df_vw_cell_defect = read_feather.read(os.path.join(GeneralConfig.path2data.value,"VW","VW_ID3_JB_8A_CEE7_GB_2024_cell_defect.feather"))

## Figure

In [ ]:
outline=mpe.withStroke(linewidth=2, foreground='black')

In [ ]:
def myround(x, prec=2, base=.05):
    return round(base * round(float(x)/base),prec)

In [ ]:
def plot_model2model(axes4):
    sec_ax = axes4.twinx()
    axes4.set_zorder(sec_ax.get_zorder()+1)  # default zorder is 0 for ax1 and ax2
    axes4.patch.set_visible(False) 
    
    axes4.grid()
    axes4.set_axisbelow(True)
   
    axes4.plot(df_cupra_397["SOC"],df_cupra_397["U"],color=colors[0],path_effects=[outline],label="Cupra #1")
    axes4.plot(df_cupra_349["SOC"],df_cupra_349["U"],color=colors[1],path_effects=[outline],label="Cupra #2")
    axes4.plot(df_cupra_288["SOC"],df_cupra_288["U"],color=colors[2],path_effects=[outline],label="Cupra #3")
    axes4.plot(df_cupra_204["SOC"],df_cupra_204["U"],color=colors[3],path_effects=[outline],label="Cupra #4")
    axes4.plot(df_cupra_213["SOC"],df_cupra_213["U"],color=colors[4],path_effects=[outline],label="Cupra #5")
    axes4.set_xlim([0,100])
    tick_positions = np.array([0,25,50,75,100])
    axes4.set_xticks(tick_positions)
    axes4.set_ylim([360,460])
    tick_positions = np.array([360,410,460])
    axes4.set_yticks(tick_positions)
    axes4.yaxis.set_minor_locator(FixedLocator([385,435]))
    axes4.set_ylabel("Pack voltage in V")
    # axes4.legend(loc="upper left", framealpha =1,ncols=1, fancybox=False,labelspacing=0.1,columnspacing=1,edgecolor="black",fontsize=6)
    axes4.set_xlabel("BMS-SOC in %")
    #axes4.set_xticklabels([])
    handles, labels =axes4.get_legend_handles_labels() 
    patch = patches.Patch(facecolor=VisualizationConfig.TUMgrau20.value,edgecolor=VisualizationConfig.TUMgrau50.value, label=r'$\Delta U$')
    handles = handles+[patch]
    axes4.legend(handles=handles,loc="lower right",ncol=2, framealpha =1, fancybox=False,labelspacing=0.1,columnspacing=0.5,edgecolor="black",fontsize=6,handlelength=1.5)
    # secondary axis
    sec_ax.plot(df_U_spread["SOC"],df_U_spread["dU_min"],color=VisualizationConfig.TUMgrau50.value)
    sec_ax.plot(df_U_spread["SOC"],df_U_spread["dU_max"],color=VisualizationConfig.TUMgrau50.value)
    sec_ax.fill_between(df_U_spread["SOC"], df_U_spread["dU_min"], df_U_spread["dU_max"],color=VisualizationConfig.TUMgrau20.value)
    sec_ax.set_ylim([-8,8])
    tick_labels = np.array([-8,-4,0,4,8])
    sec_ax.set_yticks(tick_labels)
    sec_ax.yaxis.set_minor_locator(FixedLocator([-6,-2,2,6]))
    sec_ax.set_ylabel("Voltage spread to mean in V")
    #sec_ax.set_ylabel("Voltage spread in V")
    ## arrow
    axes4.annotate(text='', xy=(97,df_U_spread[df_U_spread["SOC"]>75]["dU_min"].min()*(25/4)+410), xytext=(97,df_U_spread[df_U_spread["SOC"]>75]["dU_max"].max()*(25/4)+410), arrowprops=dict(arrowstyle='<|-|>', shrinkA=0, shrinkB=0,facecolor="k",lw=1))
    #print(df_U_spread[df_U_spread["SOC"]>75]["dU_max"].max()-df_U_spread[df_U_spread["SOC"]>75]["dU_min"].min())
    axes4.annotate(text='5.4 V', xy=(97,410),horizontalalignment="right",verticalalignment="center",rotation=90,fontsize=7)
    return

In [ ]:
def plot_software(axes):
    sec_ax = axes.twinx()
    axes.set_zorder(sec_ax.get_zorder()+1)  # default zorder is 0 for ax1 and ax2
    axes.patch.set_visible(False) 
    
    axes.grid(which="both",axis="y")
    axes.set_axisbelow(True)
    axes.bar([25,50,75],[df_vw["E"].max()*(100/80)+360,df_vw_2["E"].max()*(100/80)+360,df_vw_3["E"].max()*(100/80)+360],width=13,color=[colors[0],colors[2],colors[4]],ec="k")
    axes.plot(df_vw["SOC"],df_vw["U"],color=colors[0],label="ID.Software 2.0 (2021)",lw=1,path_effects=[outline])
    axes.plot(df_vw_2["SOC"],df_vw_2["U"],color=colors[2],label="ID.Software 2.4 (2023)",lw=1,path_effects=[outline])
    axes.plot(df_vw_3["SOC"],df_vw_3["U"],color=colors[4],label="ID.Software 3.2 (2024)",lw=1,path_effects=[outline])

    axes.set_xlim([-0.1,100])
    tick_positions = np.array([0,25,50,75,100])
    axes.set_xticks(tick_positions)
    axes.set_ylim([360,460])
    tick_positions = np.array([360,410,460])
    axes.set_yticks(tick_positions)
    axes.yaxis.set_minor_locator(FixedLocator([385,435]))
    axes.set_ylabel("Pack voltage in V")
    axes.legend(loc="lower right", framealpha =1,ncols=1, fancybox=False,labelspacing=0.1,columnspacing=1,edgecolor="black",fontsize=6)
    axes.set_xlabel("BMS-SOC in %")
    
    sec_ax.bar([25,50,75],[df_vw["E"].max(),df_vw_2["E"].max(),df_vw_3["E"].max()],width=13,color=None,alpha=0.7)
    sec_ax.set_ylabel("Charged energy in kWh")
    sec_ax.set_xlim([-0.1,100])
    sec_ax.set_ylim([0,80])
    tick_positions = np.array([0,20,40,60,80])
    sec_ax.set_yticks(tick_positions)
    #sec_ax.yaxis.set_minor_locator(FixedLocator([50,150]))
    
    axes.annotate(text=f'{df_vw.E.max():.1f}', xy=(25,df_vw["E"].max()*(100/80)+360+7),fontsize=7,rotation=0,ha="center",va="center",bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.7))
    axes.annotate(text=f'{df_vw_2.E.max():.1f}', xy=(50,df_vw_2["E"].max()*(100/80)+360+7),fontsize=7,rotation=0,ha="center",va="center",bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.7))
    axes.annotate(text=f'{df_vw_3.E.max():.1f}', xy=(75,df_vw_3["E"].max()*(100/80)+360+7),fontsize=7,rotation=0,ha="center",va="center",bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.7))
    return

In [ ]:
def plot_balance_volt(axes):
    sec_ax = axes.twinx()
    axes.set_zorder(sec_ax.get_zorder()+1)  # default zorder is 0 for ax1 and ax2
    axes.patch.set_visible(False) 
    
    axes.grid()
    axes.set_axisbelow(True)
    axes.plot(df_vw_relax["time_h"],df_vw_relax["I"].rolling(12).mean()*(100/16)+ 385,color=colors[1],linewidth=1,alpha=1,linestyle="--")
    axes.plot(df_vw_balance["time_h"],df_vw_balance["I"].rolling(12).mean()*(100/16) + 385,color=colors[1],linewidth=1,alpha=1,)
    
    axes.plot(df_vw_relax["time_h"],df_vw_relax["U"],color=colors[4],label="",linestyle="--")
    axes.plot(df_vw_balance["time_h"],df_vw_balance["U"],color=colors[4],alpha=1,linestyle="-")


    axes.set_xlim([0,60])
    tick_positions = np.array([0,10,20,30,40,50,60])
    axes.set_xticks(tick_positions)
    axes.set_ylim([360,460])
    tick_positions = np.array([360,410,460])
    axes.set_yticks(tick_positions)
    axes.yaxis.set_minor_locator(FixedLocator([385,435]))
    axes.set_ylabel("Pack voltage in V")
    #axes.legend(loc="lower right", framealpha =1,ncols=1, fancybox=False,labelspacing=0.1,columnspacing=1,edgecolor="black",fontsize=6)
    axes.set_xlabel("Time in h")

     # legend
    handles, labels =axes.get_legend_handles_labels()  
    line = plt.hlines(-100,0,1,label='Voltage ', color=colors[4])
    line1 = plt.hlines(-100,0,1,label='Current', color=colors[1])
    line2 = plt.hlines(-100,0,1,label='Imbalanced cells', color="k")
    line3 = plt.hlines(-100,0,1,label='Balanced cells', color="k",linestyle="--")
    # add manual symbols to auto legend
    handles.extend([line,line1, line3,line2])
    axes.legend(handles=handles,loc="lower right",ncols=2,handletextpad=0.4,fontsize=6,
               framealpha =1 , fancybox=False,labelspacing=0.1,columnspacing=1,edgecolor="black",handlelength=1.8)
      
    #sec_ax.plot(df_vw_relax["time_h"]-24.5-12,df_vw_relax["I"].rolling(2).mean(),color=colors[1],linewidth=1,alpha=0)
    #sec_ax.plot(df_vw_balance["time_h"]-12,df_vw_balance["I"].rolling(2).mean(),color=colors[4],linewidth=1,alpha=0)
    sec_ax.set_xlim([0,60])
    sec_ax.set_ylim([-4,12])
    tick_positions = np.array([-4,0,4,8,12])
    #sec_ax.yaxis.set_minor_locator(FixedLocator([-3,3]))
    sec_ax.set_yticks(tick_positions)
    sec_ax.set_ylabel("Charging current in A")

    #annotations
    axes.annotate(text='passive balancing\n after charging', xy=(50,412),horizontalalignment="center",verticalalignment="bottom",fontsize=6,bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.8))
    axes.annotate(text='', xy=(38,390), xytext=(50,412), arrowprops=dict(arrowstyle='-|>', shrinkA=5, shrinkB=5,facecolor="k"))

    axes.annotate(text='100% SOC\n displayed in UI', xy=(20,440),horizontalalignment="center",verticalalignment="bottom",fontsize=6,bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.8))
    axes.annotate(text='', xy=(37,455), xytext=(27,445), arrowprops=dict(arrowstyle='-|>', shrinkA=0, shrinkB=5,facecolor="k"))
    axes.annotate(text='', xy=(40,447), xytext=(27,445), arrowprops=dict(arrowstyle='-|>', shrinkA=0, shrinkB=5,facecolor="k"))
    return

In [ ]:
def plot_cell_defect(axes):
    sec_ax = axes.twinx()
    axes.set_zorder(sec_ax.get_zorder()+1)  # default zorder is 0 for ax1 and ax2
    axes.patch.set_visible(False) 
    
    axes.grid()
    axes.set_axisbelow(True)
    
    cell_cols = [col for col in df_vw_cell_defect.columns if "cell" in col]
    for cell in cell_cols[::-1]:
        axes.plot(df_vw_cell_defect["SOC"],df_vw_cell_defect[cell]*108,color = VisualizationConfig.TUMblau.value,linewidth=1)
    axes.plot(df_vw_cell_defect["SOC"],df_vw_cell_defect[cell_cols[-1]]*108,color = VisualizationConfig.TUMblau.value,linewidth=1,label="Cells")
    axes.plot(df_vw_cell_defect["SOC"],df_vw_cell_defect["U"],color=VisualizationConfig.TUMorange.value,linewidth=1.5,label="Pack")
    defect=13
    axes.plot(df_vw_cell_defect["SOC"],df_vw_cell_defect[cell_cols[defect]]*108,color = VisualizationConfig.TUMgrau50.value,linewidth=1,path_effects=[outline],label="Defective cell 13")
    
    axes.set_xlim([0,100])
    tick_positions = np.array([0,25,50,75,100])
    axes.set_xticks(tick_positions)
    axes.set_ylim([360,460])
    tick_positions = np.array([360,410,460])
    axes.set_yticks(tick_positions)
    axes.set_ylabel("Pack voltage in V")
    axes.yaxis.set_minor_locator(FixedLocator([385,435]))
    axes.set_xlabel("BMS-SOC in %")
    
    axes.legend(loc="lower right", framealpha =1, fancybox=False,labelspacing=0.1,columnspacing=1,edgecolor="black",fontsize=6)

    
    #sec_ax.set_xlim([0,160])
    sec_ax.set_ylim(np.array([360,460])/106)
    tick_labels = np.array([360,410,460])/106
    sec_ax.set_yticks(tick_labels)
    sec_ax.set_yticklabels([f"{myround(label):.2f}" for label in tick_labels])
    sec_ax.set_ylabel("Cell voltage in V")
    sec_ax.yaxis.set_minor_locator(FixedLocator([385/106,435/106]))

    # # inset axes left
    axins = axes.inset_axes([6, 412,46,44],transform = axes.transData) # [x0, y0, width, height]
    for cell in cell_cols[::-1]:
        axins.plot(df_vw_cell_defect["SOC"],df_vw_cell_defect[cell]*108,color = VisualizationConfig.TUMblau.value,linewidth=1)
    axins.plot(df_vw_cell_defect["SOC"],df_vw_cell_defect[cell_cols[-1]]*108,color = VisualizationConfig.TUMblau.value,linewidth=1,label="Cells")
    axins.plot(df_vw_cell_defect["SOC"],df_vw_cell_defect["U"],color=VisualizationConfig.TUMorange.value,linewidth=1.5)
    defect=13
    axins.plot(df_vw_cell_defect["SOC"],df_vw_cell_defect[cell_cols[defect]]*108,color = VisualizationConfig.TUMgrau50.value,linewidth=1,path_effects=[outline])
    # subregion of the original image
    x1, x2, y1, y2 = 0, 25,362,390
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)
    axins.set_xticklabels([])
    axins.set_yticklabels([])
    #axins.set_yticks([440,445,450,455])
    axins.set_yticks([])
    axins.set_xticks([])
    axins.grid()
    axes.indicate_inset_zoom(axins, edgecolor="black")
    return

In [ ]:
fig = plt.figure(constrained_layout=True,figsize=(cm2inch(VisualizationConfig.textwidth_in_cm.value),
                                                3*cm2inch(3)))
gs= fig.add_gridspec(nrows=2, ncols=2,width_ratios=[1,1])

axes1 = fig.add_subplot(gs[0, 0])
plot_model2model(axes1)

axes2 = fig.add_subplot(gs[0, 1])
plot_software(axes2)

axes3 = fig.add_subplot(gs[1, 0])
plot_balance_volt(axes3)

axes4= fig.add_subplot(gs[1,1])
plot_cell_defect(axes4)
#plt.tight_layout()
axes1.text(-0.25, 1, 'a)', horizontalalignment='left',fontsize=7,
     verticalalignment='center', transform=axes1.transAxes)
axes2.text(-0.25, 1, r'b)', horizontalalignment='left',fontsize=7,
     verticalalignment='center', transform=axes2.transAxes)
axes3.text(-0.25, 1, r'c)', horizontalalignment='left',fontsize=7,
     verticalalignment='center', transform=axes3.transAxes)
axes4.text(-0.25, 1, r'd)', horizontalalignment='left',fontsize=7,
     verticalalignment='center', transform=axes4.transAxes)

fig.savefig(os.path.join(GeneralConfig.path2figures.value,"20-POCV_discussion.pdf"))
fig.savefig(os.path.join(GeneralConfig.path2figures.value,"20-POCV_discussion.png"),dpi=300, pad_inches = 0)